<div style="direction:ltr; line-height:300%; font-size:20px;">
  <div style="text-align:center; font-size:20px;">
    <br>
    Advanced Information Retrieval Course
    <br><br>
    <b>Phase 1: Data Acquisition and Indexing Infrastructure</b>
    <br><br>
    Sharif University of Technology
    <br>
    Department of Computer Engineering
    <br><br>
  </div>
</div>
<div dir="ltr"></div>

# 🎯 Introduction

Welcome to your project! 

### Key Points:
- **3 Phases**: Step-by-step progress.
- **Dataset**: Scientific articles from [Semantic Scholar](https://www.semanticscholar.org/).
- **Dataset Category**: `Artificial Intelligence & Bioinformatics`.

By focusing on one domain, you'll boost the quality of your retrieval system.

# 🛠️ Phase 1: Building the Foundation

The first phase is dedicated to laying the groundwork for a robust information retrieval system. This involves creating a reliable infrastructure for data processing and indexing, which ensures efficient and accurate retrieval of academic documents.

### Key Components

- **📂 Data Preprocessing & Preparation:** Initial data ingestion and structuring of academic papers to facilitate efficient retrieval.
- **🔍 Positional Index:** Development of a positional index to enable fast and precise document search capabilities.
- **📊 Vector Space Models:** Implementation of various vector space models (`ltn-lnn`, `ltc-lnc`, `Okapi BM25`) for effective document ranking.
- **📝 Spell Correction:** Integration of bigram-based spell correction to enhance search accuracy.
- **📈 Performance Metrics:** Evaluation of system performance using metrics like MRR, Precision, Recall, F1 Score, MAP, and NDCG.


### 🧹 Data Preprocessing & Preparation

This step is crucial. Preprocessing your data correctly will make a significant difference in the effectiveness of your retrieval system.

#### Steps:
1. **Read Data**: Load your dataset from the file.
2. **Preprocess**: Use libraries like [SpaCy](https://spacy.io/) and [NLTK](https://www.nltk.org/).
3. **Implement**: Create the `clean_data()` function:
   - **Input**: Text (title or abstract).
   - **Output**: List of valid tokens after lemmatization, stemming, and case folding.
4. **Clean Up**: Remove punctuation.
5. **Stop Words**: Identify and handle as instructed.

In [1]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from string import punctuation


def clean_data(text : str):
    """Preprocesses the text with tokenization, case folding, stemming and lemmatization, and punctuations

    Parameters
    ----------
    text : str
        The title or abstract of an article

    Returns
    -------
    list
        A list of tokens
    """    

    text = text.lower()    
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in punctuation]
    stemmer = SnowballStemmer(language='english')
    lemmatizer = WordNetLemmatizer()
    preprocessed_tokens = []
    
    for token in tokens:
        lemma = lemmatizer.lemmatize(token)
        stem = stemmer.stem(lemma)
        preprocessed_tokens.append(stem)
    
    return preprocessed_tokens


clean_data("An abstract is a summary of the main article.") # return ["an", "abstract", "is", "a", "summary", "of", "the", "main", "article"]

['an', 'abstract', 'is', 'a', 'summari', 'of', 'the', 'main', 'articl']

### Implementing Stop Word Removal

Next, we'll implement the `find_stop_words()` function to identify and remove unnecessary tokens (stop words). Here’s how:

- **Identify Frequent Tokens**: First, find the tokens that appear most frequently.
- **Remove Top 30 Tokens**: Eliminate the 30 most frequent tokens from the entire text dataset. These are typically stop words that add little meaning.
- **Output**: Print the stop words along with their frequency in any format.

In [2]:
import collections


def find_stop_words(all_text : list[str], num_token=20):
    """Detects stop-words

     Parameters
    ----------
    all_text : list of all tokens
        (result of clean_data(text) for all the text)

    Returns
    -------
    Return Value is optional but must print the stop words and number of their occurence
    """

    # TODO: find num_token top most repetitive terms and report them

    all_tokens = [item for sublist in all_text for item in (sublist if isinstance(sublist, list) else [sublist])]
    token_counts = collections.Counter(all_tokens)

    top_30_tokens = [token for token, count in token_counts.most_common(30)]
    top_30_tokens_count = [count for token, count in token_counts.most_common(30)]
    
    preprocessed_texts = []
    for tokens in all_text:
        filtered_tokens = [token for token in tokens if token not in top_30_tokens]
        preprocessed_texts.append(filtered_tokens)
    
    return preprocessed_texts, {key: value for key, value in zip(top_30_tokens, top_30_tokens_count)}


sample = [clean_data("An abstract is a summary of the main article.")]
find_stop_words(sample)

([[]],
 {'an': 1,
  'abstract': 1,
  'is': 1,
  'a': 1,
  'summari': 1,
  'of': 1,
  'the': 1,
  'main': 1,
  'articl': 1})

In [3]:
# TODO: read data file and do the preprocessing on the data (first clean the data and then detect and delete stop words)
import csv
import collections

def get_filtered_tokens(all_tokens, excluded_tokens):
    filtered_tokens = []
    for tokens in all_tokens:
        to_keep_tokens = [token for token in tokens if token not in excluded_tokens]
        filtered_tokens.append(to_keep_tokens)
    return filtered_tokens

all_titles = collections.defaultdict(str)
all_abstracts = collections.defaultdict(str)

all_paper_ids = []
all_abstract_tokens = []
all_title_tokens = []

with open('AI & Bioinformatics/data.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        title = row['title']
        abstract = row['abstract']
        paper_id = row['paperId']
        
        all_titles[paper_id] = title
        all_abstracts[paper_id] = abstract
        
        title_tokens = clean_data(title)
        abstract_tokens = clean_data(abstract)
        
        all_paper_ids.append(paper_id)
        all_title_tokens.append(title_tokens)
        all_abstract_tokens.append(abstract_tokens)
        
all_data = list(all_titles.values()) + list(all_abstracts.values())
all_tokens, top_30_tokens = find_stop_words(all_title_tokens + all_abstract_tokens, 10)
all_abstract_tokens = get_filtered_tokens(all_abstract_tokens, list(top_30_tokens.keys()))
all_title_tokens = get_filtered_tokens(all_title_tokens, list(top_30_tokens.keys()))

corpus = {}
for p, t, a in zip(all_paper_ids, all_title_tokens, all_abstract_tokens):
    corpus[p] = {
        'title': t,
        'abstract': a
    }

### 🗂️ Indexing

In this section, we'll build a **positional index** for the system. This index will allow for separate and differently scored searches on various parts of a document, such as the title or abstract.

- **Build the Index**: Create an index that enables search across different document sections.
- **Functionality**: For any given word, your index should allow you to find:
  - The document IDs where the word appears.
  - The exact positions of the word in each section (title and abstract).

In [4]:
from collections import defaultdict

def construct_positional_indexes(corpus):
    
    """
    Get processed data and insert words in that into a trie and construct postional_index and posting lists after wards.

    Parameters
    ----------
    corpus: str
        processed data 
    
    Return
    ----------
    docs: 
        list of docs with specificied id, title, abstract.
    """
    positional_index = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))
    frequencies = defaultdict(int)
    for paper_id, details in corpus.items():
        title, abstract = details['title'], details['abstract']
        for position, token in enumerate(title):
            positional_index[token][paper_id]['title'].append(position)
            frequencies[token] += 1
    
        for position, token in enumerate(abstract):
            positional_index[token][paper_id]['abstract'].append(position)
            frequencies[token] += 1
            
    return frequencies, positional_index
frequencies, positional_index = construct_positional_indexes(corpus)

In [5]:
title_lengths = defaultdict(int)
abstract_lengths = defaultdict(int)
for paper_id, details in corpus.items():
    title_tokens, abstract_tokens = details['title'], details['abstract']
    title_lengths[paper_id] = len(title_tokens)
    abstract_lengths[paper_id] = len(abstract_tokens)

### 🔍 Viewing the Posting List

This section allows you to view the **posting list** of a word and its positions in each document section.

- **Function `get_posting_list(word)`**:
  - **Input**: A word.
  - **Output**: A dictionary where:
    - **Keys**: Document IDs where the word appears.
    - **Values**: Another dictionary with keys `title` and `abstract`, each holding a list of positions where the word appears in the respective sections.


In [6]:
def get_posting_list(word : str):
    
    """ get posting_list of a word
    
        Parameters
        ----------
        word: str
             word we want to check

        Return
        ----------
        dict 
            posting list

    """
    posting_list = {}
    if word in positional_index:
        for paper_id in positional_index[word]:
            posting_list[paper_id] = {
                'title': positional_index[word][paper_id]['title'],
                'abstract': positional_index[word][paper_id]['abstract']
            }
            
    return posting_list

print('frequency: ', str(frequencies['analysi']))
get_posting_list('analysi')

frequency:  1779


{'40ea606185b59cd07b456cb1022d64bf41f5538d': {'title': [0], 'abstract': []},
 '22949b20aef8923833158eaf90cf7c20199fb6ec': {'title': [1], 'abstract': []},
 '07738739f377504dd0169c4f4deea9d3a9579f92': {'title': [6], 'abstract': []},
 '716c671a796970a65107b3f9bb5bf1d4349fb454': {'title': [], 'abstract': [52]},
 '9e558853e823d7449ad4ae7244a004b173f33be3': {'title': [], 'abstract': [62]},
 'f5ac55a80db574f768b6fc4f895f909281fce1da': {'title': [2], 'abstract': []},
 '2b8461b865ba1ec87b44bd32f41397e6dea20cfb': {'title': [], 'abstract': [36]},
 '0983d84234135aa07e15f50a6dcc570b5ffbfe3f': {'title': [],
  'abstract': [79, 94, 178]},
 '920b59dd282e001c8eb790b774cdcc8d2e3b7cde': {'title': [0],
  'abstract': [66, 113]},
 '8e11ef89ba00206665f39df4312154dfa75ce159': {'title': [12], 'abstract': []},
 '624061063ec313dfd9df8b96ee53ffbca91a3246': {'title': [], 'abstract': [72]},
 '1838c9215a2f8995c8723295ce3f76fec6e8e602': {'title': [], 'abstract': [125]},
 '3e0ef6ec8e3052ad9c3b3601c7fe6fad047f50ee': {'t

### 🔄 Dynamic Indexing

To make the index dynamic, you’ll add capabilities for adding and removing individual documents.

- **Add a Document**: Implement the `add_document()` function. It takes a tuple with the document ID, title, and abstract. If the document isn’t already in the index, it will be added.
- **Remove a Document**: Implement the `remove_document()` function. It takes the document ID and removes the corresponding document from the index.
- **Uniqueness**: Ensure that document IDs remain unique in the index. For instance, the same ID won’t be added twice unless it’s removed and re-added.


In [7]:
from pygtrie import StringTrie

def add_documnet(document : tuple):
    """Adds a document to positional index

    Parameters
    ----------
    document : str
        Comma separated string containing id,title,abstarct in this exact order
    """
    paper_id, title, abstract = document
    
    title_tokens = [t for t in clean_data(title) if t not in top_30_tokens.keys()]
    abstract_tokens = [t for t in clean_data(abstract) if t not in top_30_tokens.keys()]
    
    title_lengths[paper_id] = len(title_tokens)
    abstract_lengths[paper_id] = len(abstract_tokens)
    
    for i, token in enumerate(title_tokens):
        positional_index[token][paper_id]['title'].append(i)
        frequencies[token] += 1
    for i, token in enumerate(abstract_tokens):
        positional_index[token][paper_id]['abstract'].append(i)
        frequencies[token] += 1
    return

new_document = ("1eae26fe1ca566f17468080c3aecab1c3f9efb66", 
                "A Deep Learning Framework for Viable Tumor Burden Estimation", 
                "Liver masses have become a common clinical challenge since they require to be defined and accurately categorized as neoplastic or nonneoplastic lesions. Hepatocellular carcinoma (HCC), the most common histologic type of primary liver malignancy, is a global health concern being the fifth most common cancer and the second cause of cancer mortality worldwide. Accurate diagnosis, which in some circumstances requires histopathology results, is necessary for appropriate management. Also, some tumor characteristics help in predicting tumor behavior and patient response to therapy. In this paper, we propose a deep learning framework for the segmentation of whole and viable tumor areas of liver cancer from whole-slide images (WSIs). To this end, we use Fast Segmentation Convolutional Neural Network (Fast-SCNN) as our network. We use the dataset from PAIP 2019 challenge. After data-augmentation on the training subset, we train the network with a multi-term loss function and SWA technique. Our model achieves 0.80 for the median of the Jaccard Index for the task of Viable Tumor Segmentation and 0.77 for the median of Weighted Absolute Accuracy for the task of Viable Tumor Burden Estimation on the whole-slide images of the test subset.")
add_documnet(new_document)

In [8]:
def remove_document(document_id : str):
    """removes a document from positional index

    Parameters
    ----------
    document_id : str
        Id of the document
    """
    del title_lengths[document_id]
    del abstract_lengths[document_id]

    for token in list(positional_index.keys()):
        if document_id in positional_index[token]:
            token_doc_occurence = len(positional_index[token][document_id]['title']) + len(positional_index[token][document_id]['abstract'])
            del positional_index[token][document_id]
            frequencies[token] -= token_doc_occurence
    
        if not positional_index[token]:
            del positional_index[token]
        if not frequencies[token]:
            del frequencies[token]
    return

remove_document("1eae26fe1ca566f17468080c3aecab1c3f9efb66")

### 💾 Index Compression & Storage

In this section, you’ll add the ability to save and reload the index.

- **Storage Methods**: Implement three methods for saving the index:
  1. **No compression**
  2. **Gamma-code compression**
  3. **Variable-byte compression**
- **Implementation**: You’ll implement these compression techniques yourself.
- **File Format**: Save the index in JSON or TXT format. Name the file after the compression method used, and upload it in a zip file.
- **Scoring**: Part of your score here depends on the efficiency of your compression.

- **Load the Index**: Implement the `load_index(path)` function, which reloads the index from a saved file.


## Variable-Byte Method

In [9]:
import json

def encode_vbyte(num):
    bytes_list = []
    while num > 127:
        bytes_list.append(num & 127)
        num >>= 7
    bytes_list.append(num | 128)
    return bytearray(bytes_list)


def decode_vbyte(bytearray_data):
    num = 0
    for i, byte in enumerate(bytearray_data):
        if byte < 128:
            num |= byte << (7 * i)
            break
        num |= (byte & 127) << (7 * i)
    return num


def convert_gapped_to_list(gapped_arr):
    arr = []
    last_index = 0
    for i in gapped_arr:
        decoded = decode_vbyte(bytes.fromhex(i))
        arr.append(decoded + last_index)
        last_index = decoded
    return arr
        
    return gaps_arr


def convert_list_to_gaps(arr):
    gaps_arr = []
    last_index = 0
    for i in arr:
        gaps_arr.append(i - last_index)
        last_index = i
        
    return gaps_arr

def vb_encode_arr(arr):
    encoded_arr = []
    for i in arr:
        encoded_arr.append(encode_vbyte(i))
    
    return encoded_arr

def get_vb_encoded_positional_index(positional_index):
    encoded_pi = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))
    for token, papers in positional_index.items():
        for paper_id, data in papers.items():
            title_positions = data['title']
            abstarct_positions = data['abstract']
            gaps_title_positions = convert_list_to_gaps(title_positions)
            gaps_abstract_positions = convert_list_to_gaps(abstarct_positions)
            encoded_title_positions = vb_encode_arr(gaps_title_positions)            
            encoded_abstract_positions = vb_encode_arr(gaps_abstract_positions)
            encoded_pi[token][paper_id]['title'] = encoded_title_positions
            encoded_pi[token][paper_id]['abstract'] = encoded_abstract_positions
    
    return encoded_pi

def convert_gapped_positional_index_to_normal(encoded_pi):
    positional_index = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))
    for token, papers in encoded_pi.items():
        for paper_id, data in papers.items():
            gapped_title_positions = data['title']
            gapped_abstarct_positions = data['abstract']
            title_positions = convert_gapped_to_list(gapped_title_positions)
            abstract_positions = convert_gapped_to_list(gapped_abstarct_positions)
            positional_index[token][paper_id]['title'] = title_positions
            positional_index[token][paper_id]['abstract'] = abstract_positions
            
    return positional_index

class ByteArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytearray):
            return obj.hex()
        return json.JSONEncoder.default(self, obj)

## Gamma-Codes

In [10]:
def encode_gamma(num):
    """Encode an integer using Gamma codes method"""
    if num == 0:
        return "0"
    binary = bin(num)[2:]
    offset = "0" * (len(binary) - 1)
    return offset + binary

def decode_gamma(gamma_code):
    """Decode an integer encoded with Gamma codes method"""
    i = 0
    while gamma_code[i] == "0":
        i += 1
        if i == len(gamma_code):
            break
    binary = "1" + gamma_code[i+1:i*2+1]
    return int(binary, 2)



def convert_gamma_gapped_to_list(gapped_arr):
    arr = []
    last_index = 0
    for i in gapped_arr:
        decoded = decode_gamma(i)
        arr.append(decoded + last_index)
        last_index = decoded
    return arr
        
    return gaps_arr


def gamma_encode_arr(arr):
    encoded_arr = []
    for i in arr:
        encoded_arr.append(encode_gamma(int(i)))
    
    return encoded_arr

def get_gamma_encoded_positional_index(positional_index):
    encoded_pi = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))
    for token, papers in positional_index.items():
        for paper_id, data in papers.items():
            title_positions = data['title']
            abstarct_positions = data['abstract']
            gaps_title_positions = convert_list_to_gaps(title_positions)
            gaps_abstract_positions = convert_list_to_gaps(abstarct_positions)
            encoded_title_positions = gamma_encode_arr(gaps_title_positions)            
            encoded_abstract_positions = gamma_encode_arr(gaps_abstract_positions)
            encoded_pi[token][paper_id]['title'] = encoded_title_positions
            encoded_pi[token][paper_id]['abstract'] = encoded_abstract_positions
    
    return encoded_pi


def gamma_convert_gapped_positional_index_to_normal(encoded_pi):
    positional_index = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))
    for token, papers in encoded_pi.items():
        for paper_id, data in papers.items():
            gapped_title_positions = data['title']
            gapped_abstarct_positions = data['abstract']
            title_positions = convert_gamma_gapped_to_list(gapped_title_positions)
            abstract_positions = convert_gamma_gapped_to_list(gapped_abstarct_positions)
            positional_index[token][paper_id]['title'] = title_positions
            positional_index[token][paper_id]['abstract'] = abstract_positions
            
    return positional_index

In [11]:
import os
import json

def store_index(path: str, compression_type: str):
    """Stores the index in a file

    Parameters
    ----------
    path : str
        Path to store the file

    compression_type : str
        Could be one of the followings:
        - no-compression
        - gamma-code
        - variable-byte

    Returns
    int
        The size of the stored file
    """
    if compression_type == 'no-compression':
        with open(path, 'w') as f:
            json.dump(positional_index, f)

    if compression_type == 'gamma-code':
        encoded_positional_index = get_gamma_encoded_positional_index(positional_index)
        with open(path, 'w') as f:
            json.dump(encoded_positional_index, f)

    if compression_type == 'variable-byte':
        encoded_positional_index = get_vb_encoded_positional_index(positional_index)
        with open(path, 'w') as f:
            json.dump(encoded_positional_index, f, cls=ByteArrayEncoder)

    size_of_file = os.path.getsize(path)
    return size_of_file

store_index("/home/alireza/Desktop/index.json", "no-compression")

31019882

In [12]:
def load_index(path: str, compression_type: str):
    """Loads the index from a file

    Parameters
    ----------
    path : str
        Path of the file to load from

    compression_type : str
        Could be one of the followings:
        - no-compression
        - gamma-code
        - variable-byte
    """
    positional_index = None
    if compression_type == 'no-compression':
        with open(path, 'r') as f:
            positional_index = json.load(f)
    if compression_type == 'gamma-code':
        encoded_positional_index = None
        with open(path, 'r') as f:
            encoded_positional_index = json.load(f)
            positional_index = gamma_convert_gapped_positional_index_to_normal(encoded_positional_index)
    
    if compression_type == 'variable-byte':
        encoded_positional_index = None
        with open(path, 'r') as f:
            encoded_positional_index = json.load(f)
            positional_index = convert_gapped_positional_index_to_normal(encoded_positional_index)

    return positional_index

positional_index = load_index("/home/alireza/Desktop/index.json", "no-compression")

### 🔄 Spell Correction

If the user query contains spelling mistakes or words not found in the dictionary, your system should correct the query.

- **Bigram Extraction**: First, generate bigrams from the word.
- **Jaccard Similarity**: Find the top 20 words with the most common bigrams using Jaccard similarity.
- **Edit Distance**: Finally, use the minimum edit distance to find the best replacement word.

- **Note**: There’s no need to store or compress the bigram index. You can use existing code for calculating edit distance.


In [13]:
import nltk
from nltk.util import ngrams
from typing import List, Dict
import itertools


def create_bigram_index(texts: List[str]) -> Dict[str, List[str]]:
    """
    Creates a bigram index for the spell correction

    Parameters
    ----------
    texts: List[str]
        The titles and abstracts of articles

    Returns
    -------
    dict
        A dictionary of bigrams and their occurence
    """
    bigram: Dict[str, List[str]] = {}
    
    texts = [sentence.lower().split() for sentence in texts]
    tokens = list(itertools.chain(*texts))

    # TODO: Create the bigram index here
    for token in tokens:
        token = token.lower()
        token = "".join(char for char in token if char.isalnum())

        ngrams_list = list(ngrams(token, 2))
        for ngram in ngrams_list:
            if ngram not in bigram:
                bigram[ngram] = set()
            bigram[ngram].add(token)
    return bigram

bigram_index = create_bigram_index(all_data)

In [14]:
from nltk import ngrams
from nltk.metrics import jaccard_distance, edit_distance

def correct_text(input_word: str, similar_words_limit: int=20) -> str:
    """
    Correct the give query text, if it is misspelled

    Paramters
    ---------
    text: str
        The query text
    
    Returns
    str
        The corrected form of the given text
    """
    corrected_text = ""

    # TODO: Correct the text here
    input_bigrams = set(ngrams(input_word.lower(), 2))
    
    similarities = []
    for bigram in input_bigrams:
        if bigram in bigram_index:
            for word in bigram_index[bigram]:
                similarity = 1 - jaccard_distance(input_bigrams, set(ngrams(word.lower(), 2)))
                similarities.append((word, similarity))
    
    similarities.sort(key=lambda x: x[1], reverse=True)
    similar_words = [word[0] for word in similarities[:20]]
    
    try:
        corrected_word = min(similar_words, key=lambda x: edit_distance(input_word.lower(), x))
    except Exception as e:
        return [], None
    return corrected_word

corrected_word = correct_text('dep')
print(corrected_word)

deep


### 🔍 Search and Document Retrieval

In this section, you’ll implement the search functionality to retrieve documents based on a user query.

- **Search Scope**: The search should be conducted across both the title and abstract of documents.
- **Result Ranking**: Documents should be returned in order of their final scores. This score is a weighted sum of the title and abstract search scores.

- **Search Methods**:
  1. **TF-IDF (ltn-lnn and ltc-lnc variants)**: Implement these methods in vector space for ranked retrieval.
  2. **Probabilistic Retrieval (Okapi BM25)**: Implement this for probabilistic document retrieval.

- **Final Score Calculation**:
  - Use the formula:  
    `final score = weight * abstract_score + (1 - weight) * title_score`
  - The `weight` is a number between 0 and 1. If it’s 0, the title score is ignored, and vice versa.

- **Function `search(query, method, n, mode, where)`**:
  - **Input**: The search query, scoring method, number of documents to return, mode (whether to search title and abstract separately), and where (which part to search if only one).
  - **Output**: If `print=True`, return a readable list of documents with snippets.


In [15]:
def _spell_correct_query(query: str):
    splitted_lower_query = [word for word in query.lower().split() if word not in top_30_tokens]
    corrected_query_words = [correct_text(word) for word in splitted_lower_query]
    return " ".join(corrected_query_words)

def preprocess_query(query: str):
    corrected_query = _spell_correct_query(query)
    tokenized_query = [token for token in clean_data(corrected_query) if token not in top_30_tokens]
    return tokenized_query

In [16]:
import math

number_of_documents = len(corpus)
def get_idf(token, prob=False):
    df = len(positional_index[token])
    if prob:
        return max(0, math.log10((number_of_documents - df) / (df + 1)))
    else:
        return math.log10(number_of_documents / (df + 1))

def get_logarithmic_tf(tf_raw):
    if tf_raw == 0:
        return 0
    else:
        return 1 + math.log10(tf_raw)

In [17]:
import re


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def tokenize_without_change(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

def get_snippet(query, content):
    query_tokens = tokenize_without_change(query)
    content_tokens = tokenize_without_change(content)

    query_start_index = -1
    for i in range(len(content_tokens) - len(query_tokens) + 1):
        if content_tokens[i:i+len(query_tokens)] == query_tokens:
            query_start_index = i
            break

    if query_start_index == -1:
        return ' '.join(content_tokens[:20])

    start_index = max(0, query_start_index - 10)
    end_index = min(len(content_tokens), query_start_index + 10 + len(query_tokens))

    snippet_tokens = content_tokens[start_index:end_index]
    snippet = ' '.join(snippet_tokens)

    return snippet

def print_result(doc_ids, title_query, abstract_query):
    corrected_title_query = _spell_correct_query(title_query)
    abstract_query = _spell_correct_query(abstract_query)

    print('________________________________')
    for doc_id in doc_ids:
        print()
        title_content = all_titles[doc_id]
        abstract_content = all_abstracts[doc_id]
        title_snippet = get_snippet(title_query, title_content)
        abstract_snippet = get_snippet(abstract_query, abstract_content)
        print(f"Paper Id: {doc_id}")
        print(f'Title Snippet: {title_snippet}')
        print(f'Abstract Snippet: {abstract_snippet}')
        print('________________________________')



In [18]:
def get_doc_scores(tf_query, method, field='title'):
    doc_scores = defaultdict(int)
    norm_factors = defaultdict(int)

    doc_lengths = title_lengths if field == 'title' else abstract_lengths
    average_doc_length = sum([v for v in doc_lengths.values()]) / len(doc_lengths)
    for token, tf_query_raw in tf_query.items():
        if token not in positional_index:
            continue
        for doc_id, positions in positional_index[token].items():
            if len(positions[field]) == 0:
                continue
            if method == 'ltn-lnn':
                tf_doc_raw = len(positions.get(field, []))
                tf_doc = get_logarithmic_tf(tf_doc_raw)
                idf = get_idf(token)
                doc_coeff = idf * tf_doc
                
                tf_query = get_logarithmic_tf(tf_query_raw)
                doc_scores[doc_id] += tf_query * doc_coeff
                
            if method == 'ltc-lnc':
                tf_doc_raw = len(positions.get(field, []))
                tf_doc = get_logarithmic_tf(tf_doc_raw)
                idf = get_idf(token)
                doc_coeff = idf * tf_doc
                tf_query = get_logarithmic_tf(tf_query_raw)

                doc_scores[doc_id] += tf_query * doc_coeff
                norm_factors[doc_id] += doc_coeff ** 2
            if method == 'okapi25':
                k1 = 1.5
                b = 0.75
                tf_token_doc = len(positions.get(field, []))
                doc_scores[doc_id] += get_idf(token) * (k1 + 1) * tf_token_doc / (k1 * (1 - b + b * doc_lengths[doc_id] / average_doc_length) + tf_token_doc)
    
    if method == 'ltc-lnc':
        for doc_id in doc_scores:
            doc_scores[doc_id] /= math.sqrt(norm_factors[doc_id])
    
    return doc_scores

In [19]:
import collections

from typing import List, Tuple

def search(title_query: str, abstract_query: str, max_result_count: int, method: str = 'ltn-lnn', weight: float = 0.5, print=False):
    """
        Finds relevant documents to query
        
        Parameters
        ---------------------------------------------------------------------------------------------------
        max_result_count: Return top 'max_result_count' docs which have the highest scores. 
                          notice that if max_result_count = -1, then you have to return all docs
        
        mode: 'detailed' for searching in title and text separately.
              'overall' for all words, and weighted by where the word apears on.
        
        where: when mode ='detailed', when we want search query 
                in title or text not both of them at the same time.
        
        method: 'ltn-lnn' or 'ltc-lnc' or 'okapi25'

        Returns
        ----------------------------------------------------------------------------------------------------
        list
        Retreived documents with snippet
    """
    # TODO: retun top 'max_result_count' documents for your searched query
    tf_query_title = defaultdict(int)
    tf_query_abstract = defaultdict(int)
    
    tokens_title = preprocess_query(title_query) if title_query else []
    tokens_abstract = preprocess_query(abstract_query) if abstract_query else []
    query_tokens = tokens_title + tokens_abstract

    scores = collections.defaultdict(list)
    for token in tokens_title:
        tf_query_title[token] += 1
            
    for token in tokens_abstract:
        tf_query_abstract[token] += 1
    
    
    title_query_doc_scores = get_doc_scores(tf_query_title, method, 'title')
    abstract_query_doc_scores = get_doc_scores(tf_query_abstract, method, 'abstract')
    
    total_doc_ids = list(set(list(title_query_doc_scores.keys()) + list(abstract_query_doc_scores.keys())))
    doc_scores = collections.defaultdict(int)
    
    for doc_id in total_doc_ids:
        title_score = title_query_doc_scores.get(doc_id, 0)
        abstract_score = abstract_query_doc_scores.get(doc_id, 0)
        
        doc_scores[doc_id] = weight * title_score + (1 - weight) * abstract_score
        
    sorted_doc_ids = [key for key, value in sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)]
    applicable_doc_ids = sorted_doc_ids[:max_result_count]
    if print:
        print_result(applicable_doc_ids, title_query, abstract_query)
    return applicable_doc_ids
results = search('Translation Model Based on Deep Learning','Translation Model Based on Deep Learning', 10, method='ltc-lnc', print=True)

________________________________

Paper Id: 20f77d34ab1aec7d0a6e613a740aff7ec7fbf55a
Title Snippet: a machine translation framework based on neural network deep learning from semantics to feature analysis
Abstract Snippet: this paper us an framework based on semantic to feature analysis to construct a neural machine translation model let the
________________________________

Paper Id: 0ca95a34c0a59132e7f0925b189de884e51a93a6
Title Snippet: design of intelligent recognition english translation model based on deep learning
Abstract Snippet: nowadays the intercommunication and translation of global language ha become an indispensable condition for friendly communication among human being around
________________________________

Paper Id: 1167a9431e3743e16ab7e38bf60bdf63a278f7e9
Title Snippet: english translation model based on intelligent recognition and deep learning
Abstract Snippet: aiming at the problem of low accuracy of english phrase part of speech recognition poor english translat

### 🧪 Evaluation

This section focuses on evaluating the system's performance using a validation set.

- **Validation Data**: You’re provided with sample queries and their corresponding documents in a validation file. Each query has the necessary details for creation.
- **Process**:
  - Generate queries based on the provided information.
  - Treat the results as `predicted results`.
  - The provided list of IDs will be your `actual results`.

- **Metrics**:
  - Implement the following evaluation metrics (without using pre-built functions):
  - Report these metrics for each query and overall.
      - MRR (Mean Reciprocal Rank)
      - Precision
      - Recall
      - F1 Score
      - MAP (Mean Average Precision)
      - NDCG (Normalized Discounted Cumulative Gain)


In [20]:
import json

def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data
query_ground_truth = read_json_file('./AI & Bioinformatics/validation.json')

In [21]:
from typing import List


def calculate_precision(actual: List[List[str]], predicted: List[List[str]]) -> float:
    """
    Calculates the precision of the predicted results

    Parameters
    ----------
    actual : List[List[str]]
        The actual results
    predicted : List[List[str]]
        The predicted results

    Returns
    -------
    float
        The precision of the predicted results
    """
    precision = 0.0

    # TODO: Calculate precision here
    num_retrieved = 0
    num_relevant = 0
    
    for i in range(len(actual)):
        actual_set = set(actual[i])
        retrieved_set = set(predicted[i])
        
        num_retrieved += len(retrieved_set)
        num_relevant += len(actual_set.intersection(retrieved_set))
    
    if num_retrieved == 0:
        precision = 0.0
    else:
        precision = num_relevant / num_retrieved
    
    return precision

In [22]:
def calculate_recall(actual: List[List[str]], predicted: List[List[str]]) -> float:
    """
    Calculates the recall of the predicted results

    Parameters
    ----------
    actual : List[List[str]]
        The actual results
    predicted : List[List[str]]
        The predicted results

    Returns
    -------
    float
        The recall of the predicted results
    """
    num_retrieved = 0
    num_relevant = 0
    
    for i in range(len(actual)):
        actual_set = set(actual[i])
        retrieved_set = set(predicted[i])
        
        num_retrieved += len(retrieved_set)
        num_relevant += len(actual_set.intersection(retrieved_set))
    
    if num_relevant == 0:
        recall = 0.0
    else:
        recall = num_relevant / sum(len(docs) for docs in actual_docs)
    
    return recall

In [23]:
def calculate_F1(actual: List[List[str]], predicted: List[List[str]]) -> float:
    """
    Calculates the F1 score of the predicted results

    Parameters
    ----------
    actual : List[List[str]]
        The actual results
    predicted : List[List[str]]
        The predicted results

    Returns
    -------
    float
        The F1 score of the predicted results    
    """
    f1 = 0.0

    # TODO: Calculate F1 here
    precision = calculate_precision(actual, predicted)
    recall = calculate_recall(actual, predicted)

    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return f1

In [24]:
def calculate_MAP(actual: List[List[str]], predicted: List[List[str]]) -> float:
    """
    Calculates the Mean Average Precision of the predicted results

    Parameters
    ----------
    actual : List[List[str]]
        The actual results
    predicted : List[List[str]]
        The predicted results

    Returns
    -------
    float
        The Mean Average Precision of the predicted results
    """
    
    def average_precision(actual, predicted):
        precision = []
        relevant_count = 0
        for i, item in enumerate(predicted):
            if item in actual:
                relevant_count += 1
                precision.append(relevant_count / (i + 1))
        if len(precision) == 0:
            return 0
        return sum(precision) / len(precision)

    ap = []
    for actual_i, predicted_i in zip(actual, predicted):
        ap.append(average_precision(actual_i, predicted_i))
    return sum(ap) / len(ap)

In [29]:
import numpy as np
def cacluate_NDCG(actual: List[List[str]], predicted: List[List[str]]) -> float:
    """
    Calculates the Normalized Discounted Cumulative Gain (NDCG) of the predicted results

    Parameters
    ----------
    actual : List[List[str]]
        The actual results
    predicted : List[List[str]]
        The predicted results

    Returns
    -------
    float
        The NDCG of the predicted results
    """
    ndcg = 0.0
    k = 10
    for actual_i, predicted_i in zip(actual, predicted):
        if k is not None:
            actual_i = actual_i[:k]
            predicted_i = predicted_i[:k]
        rel = [1 if item in actual_i else 0 for item in predicted_i]
        dcg = np.sum((2**np.asarray(rel) - 1) / np.log2(np.arange(2, len(rel) + 2)))
        idcg = np.sum((2**np.sort(rel)[::-1] - 1) / np.log2(np.arange(2, len(rel) + 2)))
        ndcg += dcg / idcg
    return ndcg / len(actual)
    return ndcg_score

In [30]:
def cacluate_MRR(actual: List[List[str]], predicted: List[List[str]]) -> float:
    """
    Calculates the Mean Reciprocal Rank of the predicted results

    Parameters
    ----------
    actual : List[List[str]]
        The actual results
    predicted : List[List[str]]
        The predicted results

    Returns
    -------
    float
        The MRR of the predicted results
    """
    mrr_score = 0.0
    num_queries = len(actual)
    
    for i in range(num_queries):
        query_actual_docs = set(actual[i])
        query_retrieved_docs = predicted[i]
        num_retrieved = len(query_retrieved_docs)
        
        for j in range(num_retrieved):
            if query_retrieved_docs[j] in query_actual_docs:
                mrr_score += 1.0 / (j+1)
                break
    
    mrr_score /= num_queries
    
    return mrr_score

In [31]:
actual_docs = list(query_ground_truth.values())
queries = list(query_ground_truth.keys())

def predict(queries, method, weight=0.5):
    predicted = []
    for query in queries:
        predicted.append(
            search(
                query,
                query,
                10,
                weight=weight,
                method=method,
            )
        )
    return predicted

In [32]:
# TODO: Call evaluation functions here and report the results for ltn.lnn
ltn_lnn_predicted = predict(queries, 'ltn-lnn')

print(f"ltn.lnn precision = {calculate_precision(actual_docs, ltn_lnn_predicted)}")
print(f"ltn.lnn recall = {calculate_recall(actual_docs, ltn_lnn_predicted)}")
print(f"ltn.lnn F1 = {calculate_F1(actual_docs, ltn_lnn_predicted)}")
print(f"ltn.lnn MAP = {calculate_MAP(actual_docs, ltn_lnn_predicted)}")
print(f"ltn.lnn NDCG = {cacluate_NDCG(actual_docs, ltn_lnn_predicted)}")
print(f"ltn.lnn MRR = {cacluate_MRR(actual_docs, ltn_lnn_predicted)}")


ltn.lnn precision = 0.6
ltn.lnn recall = 0.6
ltn.lnn F1 = 0.6
ltn.lnn MAP = 0.7147791215251532
ltn.lnn NDCG = 0.8026826478824044
ltn.lnn MRR = 0.7321428571428571


In [33]:
# TODO: Call evaluation functions here and report the results for ltc.lnc
ltc_lnc_predicted = predict(queries, 'ltc-lnc')

print(f"ltc.lnc precision = {calculate_precision(actual_docs, ltc_lnc_predicted)}")
print(f"ltc.lnc recall = {calculate_recall(actual_docs, ltc_lnc_predicted)}")
print(f"ltc.lnc F1 = {calculate_F1(actual_docs, ltc_lnc_predicted)}")
print(f"ltc.lnc MAP = {calculate_MAP(actual_docs, ltc_lnc_predicted)}")
print(f"ltc.lnc NDCG = {cacluate_NDCG(actual_docs, ltc_lnc_predicted)}")
print(f"ltc.lnc MRR = {cacluate_MRR(actual_docs, ltc_lnc_predicted)}")

ltc.lnc precision = 0.7
ltc.lnc recall = 0.7
ltc.lnc F1 = 0.7
ltc.lnc MAP = 0.8688964474678761
ltc.lnc NDCG = 0.9289264934141447
ltc.lnc MRR = 0.9166666666666666


In [34]:
# TODO: Call evaluation functions here and report the results for Okapi BM25
okapi25_predicted = predict(queries, 'okapi25')

print(f"Okapi BM25 precision = {calculate_precision(actual_docs, okapi25_predicted)}")
print(f"Okapi BM25 recall = {calculate_recall(actual_docs, okapi25_predicted)}")
print(f"Okapi BM25 F1 = {calculate_F1(actual_docs, okapi25_predicted)}")
print(f"Okapi BM25 MAP = {calculate_MAP(actual_docs, okapi25_predicted)}")
print(f"Okapi BM25 NDCG = {cacluate_NDCG(actual_docs, okapi25_predicted)}")
print(f"Okapi BM25 MRR = {cacluate_MRR(actual_docs, okapi25_predicted)}")


Okapi BM25 precision = 0.65
Okapi BM25 recall = 0.65
Okapi BM25 F1 = 0.65
Okapi BM25 MAP = 0.720045325228857
Okapi BM25 NDCG = 0.8169236925370013
Okapi BM25 MRR = 0.75
